# 📤 Entregar Trabajo

**Instrucciones:**
1. Completa todos los ejercicios en este notebook
2. Ejecuta todas las celdas
3. Ejecuta las 2 celdas de abajo para entregar

**NO requiere cuenta ni login** - solo ingresa tu nombre y listo.

In [ ]:
import json

# Check if we're in Pyodide (browser environment)
try:
    import pyodide
    print("✅ Pyodide detectado - Estamos en el navegador")
except ImportError:
    print("⚠️ Este notebook solo funciona en JupyterLite")
    print("   Visita: https://juakongdf.github.io/chaos-lab/")
    raise

SERVER_URL = "https://chaos-sub.vercel.app/submit"

def submit_current_notebook():
    """Envía el notebook actual al servidor usando JavaScript"""
    from pyodide.code import run_js
    
    # Get student name from input
    student_name_code = """
    document.getElementById('studentName').value.trim()
    """
    student_name = run_js(student_name_code)
    
    if not student_name:
        run_js("document.getElementById('result').innerHTML = '<div style=\"padding:10px;background:#fee;border-radius:4px;color:#d00\">❌ Por favor ingresa tu nombre</div>'")
        return
    
    run_js("document.getElementById('result').innerHTML = '<div style=\"padding:10px;background:#e7f3ff;border-radius:4px;color:#0066cc\">⏳ Enviando tu trabajo...</div>'")
    
    # Read the notebook file
    try:
        with open("99_Entregar_Trabajo.ipynb", 'r') as f:
            notebook_content = json.load(f)
    except Exception as e:
        error_msg = str(e).replace("'", "\\'").replace('"', '\\"')
        run_js(f"document.getElementById('result').innerHTML = '<div style=\"padding:10px;background:#fee;border-radius:4px;color:#d00\">❌ Error leyendo notebook: {error_msg}</div>'")
        return
    
    # Prepare payload as JSON string
    payload = {
        "student_name": student_name,
        "notebook": notebook_content
    }
    
    payload_json = json.dumps(payload).replace("'", "\\'")
    
    # Send via JavaScript fetch - properly escaped
    js_code = f"""
    (async function() {{
        try {{
            const payload = {payload_json};
            
            const response = await fetch('{SERVER_URL}', {{
                method: 'POST',
                headers: {{
                    'Content-Type': 'application/json',
                }},
                body: JSON.stringify(payload)
            }});
            
            const data = await response.json();
            
            if (response.ok) {{
                document.getElementById('result').innerHTML = 
                    '<div style="padding:15px;background:#d4edda;border-radius:4px;color:#155724;border-left:4px solid #28a745">' +
                    '<strong>✅ ¡Trabajo entregado exitosamente!</strong><br>' +
                    '<small>Archivo: ' + data.filename + '</small></div>';
            }} else {{
                document.getElementById('result').innerHTML = 
                    '<div style="padding:10px;background:#fee;border-radius:4px;color:#d00">❌ Error: ' + (data.error || 'Error desconocido') + '</div>';
            }}
        }} catch (error) {{
            console.error('Error completo:', error);
            document.getElementById('result').innerHTML = 
                '<div style="padding:10px;background:#fee;border-radius:4px;color:#d00">❌ Error de conexión: ' + error.message + '</div>';
        }}
    }})();
    """
    
    run_js(js_code)
    print("📤 Solicitud enviada al servidor...")

print("✅ Sistema configurado")
print("👇 Ejecuta la celda de abajo para mostrar el formulario")

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create input and button widgets
name_input = widgets.Text(
    placeholder='Tu nombre completo',
    description='Nombre:',
    style={'description_width': 'initial'}
)

submit_button = widgets.Button(
    description='🚀 Entregar',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

output = widgets.Output()

def on_submit_clicked(b):
    with output:
        clear_output()
        
        student_name = name_input.value.strip()
        
        if not student_name:
            print("❌ Por favor ingresa tu nombre")
            return
        
        print("⏳ Enviando tu trabajo...")
        
        # Read the notebook file
        try:
            with open("99_Entregar_Trabajo.ipynb", 'r') as f:
                notebook_content = json.load(f)
        except Exception as e:
            print(f"❌ Error leyendo notebook: {e}")
            return
        
        # Prepare payload
        payload = {
            "student_name": student_name,
            "notebook": notebook_content
        }
        
        # Send to server using pure Python
        import urllib.request
        import urllib.error
        
        try:
            data = json.dumps(payload).encode('utf-8')
            req = urllib.request.Request(
                SERVER_URL,
                data=data,
                headers={'Content-Type': 'application/json'}
            )
            
            with urllib.request.urlopen(req) as response:
                result = json.loads(response.read().decode('utf-8'))
                print(f"✅ ¡Trabajo entregado exitosamente!")
                print(f"📄 Archivo: {result['filename']}")
                
        except urllib.error.HTTPError as e:
            error_data = json.loads(e.read().decode('utf-8'))
            print(f"❌ Error: {error_data.get('error', 'Error desconocido')}")
        except Exception as e:
            print(f"❌ Error de conexión: {e}")

submit_button.on_click(on_submit_clicked)

# Display the widgets
print("📝 Entregar Trabajo")
print("─" * 50)
display(name_input)
display(submit_button)
display(output)